Splitting the dataset inorder to same them in their directories

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [2]:
def split_dataset(dataset_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Create output directories for train, val, and test splits
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(output_dir, split)
        os.makedirs(split_dir, exist_ok=True)

    # Iterate through each class folder
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Get all file paths for the current class
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

        # Split data into train, validation, and test sets
        train_files, temp_files = train_test_split(files, test_size=(1 - train_ratio), random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=test_ratio / (val_ratio + test_ratio), random_state=42)

        # Copy files to their respective directories
        for file_list, split in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
            split_class_dir = os.path.join(output_dir, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)
            for file in file_list:
                shutil.copy(file, os.path.join(split_class_dir, os.path.basename(file)))

    print(f"Dataset split complete. Saved in {output_dir}")

In [3]:
dataset_dir = '/content/drive/MyDrive/categorized images'
output_dir = '/content/drive/MyDrive/split dataset face tone'
split_dataset(dataset_dir, output_dir)

Dataset split complete. Saved in /content/drive/MyDrive/split dataset face tone


Preprocessing Images

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [8]:
data_dir = "/content/drive/MyDrive/split dataset face tone"
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [9]:
# Image dimensions and batch size
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

In [10]:
# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3928 images belonging to 3 classes.


In [13]:
val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 843 images belonging to 3 classes.


In [14]:
test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 844 images belonging to 3 classes.


Model Building

In [15]:
from tensorflow.keras import layers, models

In [16]:
# Build a CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 classes: warm, cool, neutral
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Training the Model

In [19]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    verbose=1
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/123 ━━━━━━━━━━━━━━━━━━━━ 83s 582ms/step - accuracy: 0.9174 - loss: 0.2761 - val_accuracy: 0.9810 - val_loss: 0.0675
Epoch 2/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 69s 521ms/step - accuracy: 0.9749 - loss: 0.0979 - val_accuracy: 0.9644 - val_loss: 0.1208
Epoch 3/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 71s 542ms/step - accuracy: 0.9759 - loss: 0.0938 - val_accuracy: 0.9810 - val_loss: 0.0588
Epoch 4/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 79s 530ms/step - accuracy: 0.9849 - loss: 0.0486 - val_accuracy: 0.9858 - val_loss: 0.0588
Epoch 5/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 81s 522ms/step - accuracy: 0.9869 - loss: 0.0424 - val_accuracy: 0.9905 - val_loss: 0.0392
Epoch 6/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 83s 532ms/step - accuracy: 0.9907 - loss: 0.0377 - val_accuracy: 0.9917 - val_loss: 0.0382
Epoch 7/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 82s 535ms/step - accuracy: 0.9886 - loss: 0.0405 - val_accuracy: 0.9858 - val_loss: 0.0605
Epoch 8/10
123/123 ━━━━━━━━━━━━━━━━━━━━ 68s 520ms/step - accuracy: 0.9898 - loss: 0.0321 - val

Evaluating the model

In [20]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")

27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.9946 - loss: 0.0189
Test Accuracy: 0.99


Prediction on new Image

In [21]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [22]:
def predict_image(model, img_path):
  img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
  img_array = image.img_to_array(img) / 255.0  # Normalize
  img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

  predictions = model.predict(img_array)
  class_idx = np.argmax(predictions)
  class_labels = list(train_data.class_indices.keys())
  return class_labels[class_idx]

In [23]:
img_path = "/content/passport_sizephoto (1).jpg"
predicted_label = predict_image(model, img_path)
print(f"The predicted color tone is: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 836ms/step
The predicted color tone is: warm


Saving the model

In [24]:
# Save model in HDF5 format
model.save("color_tone_model.h5")